In [10]:
# Sewage-Treatment-Works
# Simple GIS plot of STW and used capacity 
# Written by Diran Edwards 05/01/2025
# Generates an interactive HTML


# Install required libraries
#!pip install folium geopandas pandas shapely requests

#Data source https://www.data.gov.uk/dataset/d7e2c57b-110a-462b-97a0-9833e7d26cc2/wastewater-treatment-in-england

# Import necessary libraries
import folium
import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point


df = pd.read_excel('UWWTR_Art15_24thOct2024.xlsx', sheet_name='T_UWWTPS')

# Check if necessary columns exist (Latitude, Longitude, Name)
print(df.columns)


# Create a dictionary to map original column names to new names
column_name_map = {
    'uwwLatitude': 'Latitude',
    'uwwLongitude': 'Longitude',
    'uwwName': 'Name',
    # Add more mappings as needed
}

# Rename the columns
df = df.rename(columns=column_name_map)

# Print new column names
print("\nNew column names:")
print(df.columns)
# Create a base map centered at a specific location (e.g., London, UK)
map_center = [51.5074, -0.1278]  # Coordinates for London, UK (adjust for your location)
my_map = folium.Map(location=map_center, zoom_start=10)

# Loop through the DataFrame and add markers for each sewage treatment plant
for index, row in df.iterrows():
    # Extract the facility name (from column 5)
    name = row.iloc[4]  # Column 5 (index 4 in 0-based indexing)
    
    # Extract latitude (from column 9) and longitude (from column 10)
    latitude = row.iloc[8]  # Column 9 (index 8 in 0-based indexing)
    longitude = row.iloc[9]  # Column 10 (index 9 in 0-based indexing)

    # Add a marker for each sewage treatment plant
    folium.Marker(
        location=[latitude, longitude],
        popup=name,  # Show the facility name in the popup when clicked
        icon=folium.Icon(color='blue', icon='cloud')  # Optional: Customize marker style
    ).add_to(my_map)



# Step 2: Create GeoDataFrame for sewage treatment plants (ensure latitude and longitude are correct columns)
# We assume the columns are 'Latitude' and 'Longitude'
geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
gdf_sewage = gpd.GeoDataFrame(df, geometry=geometry)

# Step 3: Query river data from OpenStreetMap using Overpass API
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
way["waterway"="river"](51.2, 0.5, 51.5, 0.9);
out geom;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# Print the structure of the first element to understand the data
print("Structure of the first element:")
print(data['elements'][0] if data['elements'] else "No elements found")

# Convert the river data to GeoJSON format
rivers = {
    "type": "FeatureCollection",
    "features": []
}

for element in data['elements']:
    if 'geometry' in element:
        coordinates = [[coord['lon'], coord['lat']] for coord in element['geometry']]
        rivers['features'].append({
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {}
        })
    else:
        print(f"Warning: 'geometry' not found in element: {element}")

print(f"Number of river features: {len(rivers['features'])}")


for element in data['elements']:
    coordinates = []
    for coord in element['geometry']:
        coordinates.append([coord['lat'], coord['lon']])
    rivers['features'].append({
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        },
        "properties": {}
    })

# Step 4: Create a map using Folium
# Choose the center of your map (for example, London, UK)
map_center = [51.5074, -0.1278]  # Latitude and Longitude of London
my_map = folium.Map(location=map_center, zoom_start=10)

# Add river data to the map (GeoJSON format)
folium.GeoJson(rivers, name='Rivers').add_to(my_map)

# Step 5: Add sewage treatment plants data to the map
for idx, row in gdf_sewage.iterrows():
    name = row['Name']
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Add a marker for each sewage treatment plant
    folium.Marker(
        location=[latitude, longitude],
        popup=name,
        icon=folium.Icon(color='blue', icon='cloud')
    ).add_to(my_map)

# Step 6: Save the map to an HTML file
my_map.save('sewage_treatment_map_with_rivers.html')

# Output to confirm map creation
print("Map saved as 'sewage_treatment_map_with_rivers.html'. Open it in a browser to view.")


Index(['uwwState', 'repCode', 'aggCode', 'uwwCode', 'uwwName',
       'uwwCollectingSystem', 'uwwDateClosing', 'uwwHistorie', 'uwwLatitude',
       'uwwLongitude', 'uwwNUTS', 'uwwLoadEnteringUWWTP', 'uwwCapacity',
       'uwwPrimaryTreatment', 'uwwSecondaryTreatment', 'uwwOtherTreatment',
       'uwwNRemoval', 'uwwPRemoval', 'uwwUV', 'uwwChlorination',
       'uwwOzonation', 'uwwSandFiltration', 'uwwMicroFiltration', 'uwwOther',
       'uwwSpecification', 'uwwBOD5Perf', 'uwwCODPerf', 'uwwTSSPerf',
       'uwwNTotPerf', 'uwwPTotPerf', 'uwwOtherPerf', 'uwwBadPerformance',
       'uwwAccidents', 'uwwBadDesign', 'uwwInformation',
       'uwwBODDischargeCalculated', 'uwwBODDischargeEstimated',
       'uwwBODDischargeMeasured', 'uwwBODIncomingCalculated',
       'uwwBODIncomingEstimated', 'uwwBODIncomingMeasured',
       'uwwCODDischargeCalculated', 'uwwCODDischargeEstimated',
       'uwwCODDischargeMeasured', 'uwwCODIncomingCalculated',
       'uwwCODIncomingEstimated', 'uwwCODIncomingMeasu

In [12]:
# Sewage-Treatment-Works
# Simple GIS plot of STW and used capacity 
# Written by Diran Edwards 05/01/2025
# Generates an interactive HTML







# Import required libraries
import folium
import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import branca

# Load sewage treatment plant data from Excel
df = pd.read_excel('UWWTR_Art15_24thOct2024.xlsx', sheet_name='T_UWWTPS')



# Create a dictionary to map original column names to new names
column_name_map = {
    'uwwLatitude': 'Latitude',
    'uwwLongitude': 'Longitude',
    'uwwName': 'Name',
    'uwwLoadEnteringUWWTP': 'LoadEntering',
    'uwwCapacity': 'Capacity' 
 
}

# Rename the columns
df = df.rename(columns=column_name_map)


# Calculate capacity used percentage
df['CapacityUsedPercent'] = (df['LoadEntering'] / df['Capacity']) * 100

# Print instances where input exceeds capacity
print("\nInstances where input exceeds capacity:")
print(df[df['LoadEntering'] > df['Capacity']][['Name', 'LoadEntering', 'Capacity', 'CapacityUsedPercent']])

# Function to get color based on percentage
def get_color(percent):
    cmap = plt.cm.get_cmap('RdYlGn_r')
    return mcolors.rgb2hex(cmap(percent / 100))

# Create a base map centered at Newcastle upon Tyne
map_center = [54.9783, -1.6178]  # Coordinates for Newcastle upon Tyne
my_map = folium.Map(location=map_center, zoom_start=10)  # Zoom level adjusted for city view

# Add markers for each sewage treatment plant
for index, row in df.iterrows():
    name = row['Name']
    latitude = row['Latitude']
    longitude = row['Longitude']
    capacity_used_percent = row['CapacityUsedPercent']
    
    folium.CircleMarker(
        location=[latitude, longitude],
        radius=5,
        popup=f"{name}<br>Capacity Used: {capacity_used_percent:.2f}%",
        color=get_color(capacity_used_percent),
        fill=True,
        fillColor=get_color(capacity_used_percent),
        fillOpacity=0.7
    ).add_to(my_map)

# Add color map legend
colormap = branca.colormap.LinearColormap(
    colors=['darkgreen', 'yellow', 'darkred'],
    vmin=0, vmax=100,
    caption='Capacity Used (%)'
)
colormap.add_to(my_map)

# Query river data from OpenStreetMap using Overpass API
overpass_url = "http://overpass-api.de/api/interpreter"
# overpass_query = """
# [out:json];
# way["waterway"="river"](51.2, -0.5, 51.8, 0.3);
# out geom;
# """

overpass_query = """
[out:json];
area["name"="United Kingdom"]["admin_level"="2"];
way["waterway"="river"]["name"](area);
out geom;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# Print the structure of the first element to understand the data
print("\nStructure of the first element:")
print(data['elements'][0] if data['elements'] else "No elements found")

# Convert the river data to GeoJSON format
rivers = {
    "type": "FeatureCollection",
    "features": []
}

for element in data['elements']:
    if 'geometry' in element:
        coordinates = [[coord['lon'], coord['lat']] for coord in element['geometry']]
        rivers['features'].append({
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {}
        })
    else:
        print(f"Warning: 'geometry' not found in element: {element}")

print(f"\nNumber of river features: {len(rivers['features'])}")

# Add river data to the map
folium.GeoJson(rivers, name='Rivers', style_function=lambda x: {'color': 'blue', 'weight': 2}).add_to(my_map)

# Save the map
my_map.save('sewage_treatment_map_with_capacity_and_rivers.html')
print("\nMap saved as 'sewage_treatment_map_with_capacity_and_rivers.html'")
#could use qgis and qeplanet for further work


Instances where input exceeds capacity:
Empty DataFrame
Columns: [Name, LoadEntering, Capacity, CapacityUsedPercent]
Index: []


/tmp/ipykernel_928/1862496694.py:50: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('RdYlGn_r')



Structure of the first element:
{'type': 'way', 'id': 290611, 'bounds': {'minlat': 51.3806285, 'minlon': -0.505943, 'maxlat': 51.3884426, 'maxlon': -0.4749015}, 'nodes': [1886026556, 2052490, 1476364781, 2052496, 2671236146, 2052491, 2671236143, 1464914935, 2671236191, 1464914942, 1476364831, 2052492, 1464914947, 2671236185, 1476110819, 2052494, 1464914949, 1476110844, 1476364797, 1464914948, 1464914944, 2063425, 1476364703, 2052495, 1476110845, 1476364780, 1476110826, 1476364773, 2063417, 1476110838, 1476364819, 1476110830, 1476110842, 1476364747, 1476110846, 1476110841, 1476110853, 1476110858, 1476364766, 1476110822, 1476110860, 1476110829, 1476110825, 2063418, 1476364750, 1476364740, 7167974363, 7167974362, 1476364713, 7167974364, 7167974365, 7167974366, 7167974367, 2063419, 1476364715, 1476110857, 7167974368, 7167974369, 7167974370, 7167974371, 1476364763, 7167974372, 7167974374, 7167974373, 1476110827, 1476110859, 2063421, 7167974359, 7167974358, 7167974357, 7167974360, 147611085